<a href="https://colab.research.google.com/github/gandhimonil9823/CS_GO_Analysis/blob/main/Algorithm_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
plt_kwargs = {'figsize': (10, 4)}
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn import model_selection
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier


In [10]:
from google.colab import drive
drive.mount('/content/drive')
#drive.mount(r'drive/folders/0AAxMXLC9WPNxUk9PVA')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
ls

drive/  sample_data/


In [12]:
cd 'drive/Shared drives/CS_GO_Data/processed_data_v2'

/content/drive/Shared drives/CS_GO_Data/processed_data_v2


In [13]:
ls

aggregated_data/  grenade_data.csv  Log_ROC.png   meta_data.csv
damage_data.csv   kill_data.csv     map_data.csv


# Data Preprocess for Data Mining Algorithms


In [14]:
df_esea_meta_demos =  pd.read_csv("meta_data.csv")

df_esea_meta_demos.head()

,file,map,round,start_seconds,end_seconds,winner_team,winner_side,round_type,ct_eq_val,t_eq_val
0,esea_match_13770997.dem,de_overpass,1,94.30782,160.9591,Team 2,Terrorist,PISTOL_ROUND,4300,4250
1,esea_match_13770997.dem,de_overpass,2,160.95910,279.3998,Team 2,Terrorist,ECO,6300,19400
2,esea_match_13770997.dem,de_overpass,3,279.39980,341.0084,Team 2,Terrorist,SEMI_ECO,7650,19250
3,esea_match_13770997.dem,de_overpass,4,341.00840,435.4259,Team 2,Terrorist,NORMAL,24900,23400
4,esea_match_13770997.dem,de_overpass,5,435.42590,484.2398,Team 1,CounterTerrorist,ECO,5400,20550


In [17]:
def terrorist_counterT_win(series_shifted_winner_side):
  terrorist_win = series_shifted_winner_side.apply(lambda x: 1 if x=='Terrorist' else 0)
  terrorist_win = terrorist_win.iloc[:-1]
  terrorist_win = terrorist_win.reset_index(drop=True)

  counterterrorist_win = series_shifted_winner_side.apply(lambda x: 1 if x=='CounterTerrorist' else 0)
  counterterrorist_win = counterterrorist_win.iloc[:-1]
  counterterrorist_win = counterterrorist_win.reset_index(drop=True)

  return terrorist_win, counterterrorist_win

def preprocess_meta_demos_algo(df_esea_meta_demos):

  df_original_preprocessed = df_esea_meta_demos[['round','ct_eq_val','t_eq_val']]

  unique_maps = df_esea_meta_demos["map"].unique()

  map_dataframe = pd.DataFrame()

  for map_cs_go in unique_maps:
    map_dataframe[map_cs_go] = df_esea_meta_demos["map"].apply(lambda x: 1 if x == map_cs_go else 0)

  series_one_element = pd.Series([0])

  binary_values_first_round = df_esea_meta_demos["round"].apply(lambda x : 0 if x == 1 else 1)

  binary_values_first_round = binary_values_first_round.rename("not_First_Round").reset_index(drop=True)

  series_shifted_winner_side = series_one_element.append(df_esea_meta_demos["winner_side"])

  terrorist_win, counterterrorist_win = terrorist_counterT_win(series_shifted_winner_side)

  terrorist_win = (terrorist_win * binary_values_first_round).rename('terrorist_win')

  counterterrorist_win = (counterterrorist_win * binary_values_first_round).rename('counterterrorist_win')

  previous_win_df = pd.concat([terrorist_win, counterterrorist_win])

  data_to_predict_T_CT = df_esea_meta_demos['winner_side'].apply(lambda x: 1 if x=='Terrorist' else 0)

  column_names = [df_original_preprocessed, map_dataframe, terrorist_win, counterterrorist_win, data_to_predict_T_CT]

  return pd.concat(column_names, axis = 'columns', join = 'inner')

In [18]:
df_meta_data = preprocess_meta_demos_algo(df_esea_meta_demos)
df_meta_data

,round,ct_eq_val,t_eq_val,de_overpass,de_cache,de_mirage,de_inferno,de_train,de_dust2,de_cbble,de_nuke,terrorist_win,counterterrorist_win,winner_side
0,1,4300,4250,1,0,0,0,0,0,0,0,0,0,1
1,2,6300,19400,1,0,0,0,0,0,0,0,1,0,1
2,3,7650,19250,1,0,0,0,0,0,0,0,1,0,1
3,4,24900,23400,1,0,0,0,0,0,0,0,1,0,1
4,5,5400,20550,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340328,18,4500,21950,0,0,1,0,0,0,0,0,1,0,1
340329,19,16400,26100,0,0,1,0,0,0,0,0,1,0,1
340330,20,15100,26000,0,0,1,0,0,0,0,0,1,0,1
340331,21,17550,25500,0,0,1,0,0,0,0,0,1,0,1


# Data Mining Algorithms

In [19]:
X = df_meta_data.drop('winner_side', axis = 1)[:10000]

y = df_meta_data['winner_side'][:10000]

# using train_test_split for splitting the data into four different sets with 20% for the test data set
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=1)

# scale the data
standard_scalar = StandardScaler()

X_train = standard_scalar.fit_transform(X_train)

X_test = standard_scalar.fit_transform(X_test)

In [20]:
def run_models(X_train, X_test, Y_train, Y_test):
  models = [
            ('LogReg', LogisticRegression()), 
            ('RF', RandomForestClassifier(max_depth=2, random_state=0)),
            ('KNN', KNeighborsClassifier(n_neighbors=10)),
            ('SVM', svm.SVC(kernel='linear'))
  ]

  scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted', 'roc_auc']
  dfs = []  
  results = []
  names = []
  for name, model in models:
    kfold = model_selection.KFold(n_splits=6, shuffle=True, random_state=90210)
    cv_results = model_selection.cross_validate(model, X_train, Y_train, cv=kfold, scoring=scoring)
    clf = model.fit(X_train, Y_train)
    y_pred = clf.predict(X_test)
    print(name)
    print(classification_report(Y_test, y_pred))
    results.append(cv_results)
    this_df = pd.DataFrame(cv_results)
    this_df['model'] = name
    dfs.append(this_df)
  final = pd.concat(dfs, ignore_index=True)
  return final

In [25]:
df_kfold_results = run_models(X_train, X_test, Y_train, Y_test)

df_kfold_results_average_values = df_kfold_results.groupby('model').mean()

df_kfold_results_average_values.columns = [str(col) + '_average' for col in df_kfold_results_average_values.columns]

df_kfold_results_average_values


LogReg
              precision    recall  f1-score   support

           0       0.64      0.68      0.66       993
           1       0.66      0.62      0.64      1007

    accuracy                           0.65      2000
   macro avg       0.65      0.65      0.65      2000
weighted avg       0.65      0.65      0.65      2000

RF
              precision    recall  f1-score   support

           0       0.64      0.67      0.66       993
           1       0.66      0.63      0.65      1007

    accuracy                           0.65      2000
   macro avg       0.65      0.65      0.65      2000
weighted avg       0.65      0.65      0.65      2000

KNN
              precision    recall  f1-score   support

           0       0.61      0.72      0.66       993
           1       0.66      0.54      0.59      1007

    accuracy                           0.63      2000
   macro avg       0.63      0.63      0.63      2000
weighted avg       0.63      0.63      0.63      2000

SVM
 

,fit_time_average,score_time_average,test_accuracy_average,test_precision_weighted_average,test_recall_weighted_average,test_f1_weighted_average,test_roc_auc_average
model,,,,,,,
KNN,0.054202,0.385949,0.635001,0.638693,0.635001,0.633450,0.689385
LogReg,0.026633,0.009013,0.664874,0.666321,0.664874,0.664736,0.736772
RF,0.263374,0.036497,0.655874,0.656155,0.655874,0.655866,0.730368
SVM,1.043792,0.188591,0.653874,0.654347,0.653874,0.653899,0.684386
